In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sklearn.metrics.pairwise import haversine_distances
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
import fiona
from shapely.geometry import Point
from geopy.geocoders import Nominatim

In [2]:
all_uni_locations= pd.read_csv('../data/Colleges_and_Universities.csv')
all_uni_locations

,X,Y,OBJECTID,IPEDSID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,ALIAS,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID
0,-8.209134e+06,4.953591e+06,1,190752,YESHIVA OF FAR ROCKAWAY DERECH AYSON RABBINICA...,802 HICKSVILLE RD,FAR ROCKAWAY,NY,11691,5219,...,NOT AVAILABLE,24,1,-999,43,43,1,35,15,NOT AVAILABLE
1,-8.267688e+06,4.940383e+06,2,481429,UNIVERSAL TRAINING INSTITUTE,174 JEFFERSON STREET,PERTH AMBOY,NJ,8861,4106,...,NOT AVAILABLE,-2,1,-999,176,176,2,-999,15,NOT AVAILABLE
2,-7.353945e+06,2.082890e+06,3,241739,UNIVERSIDAD ANA G. MENDEZ-CUPEY CAMPUS,1399 AVE. ANA G. MENDEZ,SAN JUAN,PR,926,2602,...,UAGM CUPEY,18,3,2780,6113,8893,2,-999,1020,NOT AVAILABLE
3,-1.006191e+07,4.664997e+06,4,455071,HEALING ARTS CENTER,"10073 MANCHESTER ROAD, SUITE 100",ST. LOUIS,MO,63122,NOT AVAILABLE,...,NOT AVAILABLE,-2,1,131,-999,131,2,-999,15,NOT AVAILABLE
4,-8.797220e+06,4.290711e+06,5,455141,AVEDA INSTITUTE-CHAPEL HILL,"201 S. ESTES DRIVE, SPACE 200 B",CHAPEL HILL,NC,27514,7001,...,NOT AVAILABLE,-2,1,-999,216,216,2,-999,25,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6554,-9.317151e+06,5.308190e+06,6555,494889,BAKER COLLEGE - FLINT,1050 WEST BRISTOL ROAD,FLINT,MI,48507,5508,...,NOT AVAILABLE,-2,-2,-999,-999,-999,-1,-999,-999,NOT AVAILABLE
6555,-1.082780e+07,4.536689e+06,6556,494898,WELLSPRING SCHOOL OF ALLIED HEALTH-WICHITA,"650 N CARRIAGE PARKWAY, STE 55",WICHITA,KS,67208,4501,...,NOT AVAILABLE,-2,1,-999,38,38,2,-999,14,NOT AVAILABLE
6556,-8.145887e+06,4.983035e+06,6557,494904,ACCESS CAREERS-ISLANDIA,"1930 VETERANS HIGHWAY, SUITE 10",ISLANDIA,NY,11749,1599,...,NOT AVAILABLE,-2,1,43,20,63,2,-999,18,NOT AVAILABLE
6557,-1.304572e+07,3.865065e+06,6558,494913,FRANCISCAN SCHOOL OF THEOLOGY - SAN DIEGO,5998 ALCALA PARK,SAN DIEGO,CA,92110,2492,...,NOT AVAILABLE,-2,-2,-999,-999,-999,-1,-999,-999,NOT AVAILABLE


In [3]:
private_public = pd.read_csv('../data/school_data/public_private.csv')
private_public 

,Unnamed: 0,Institution,State,Designation,Designation.1
0,100654,Alabama A&M University,AL,1,4-year public
1,100724,Alabama State University,AL,1,4-year public
2,102030,Bishop State Community College,AL,2,2-year public
3,101240,Gadsden State Community College,AL,2,2-year public
4,102313,H. Councill Trenholm State Community\n College,AL,2,2-year public
...,...,...,...,...,...
96,234164,Virginia Union University,VA,3,4-year private
97,234137,Virginia University of Lynchburg,VA,3,4-year private
98,237215,Bluefield State College,WV,1,4-year public
99,237899,West Virginia State University,WV,1,4-year public


In [4]:
hbcu_merged=pd.merge(private_public, all_uni_locations, left_on='Unnamed: 0', right_on='IPEDSID')
hbcu_merged

,Unnamed: 0,Institution,State,Designation,Designation.1,X,Y,OBJECTID,IPEDSID,NAME,...,ALIAS,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID
0,100654,Alabama A&M University,AL,1,4-year public,-9.636762e+06,4.134481e+06,5132,100654,ALABAMA A & M UNIVERSITY,...,AAMU,18,3,728,5249,5977,1,3220,918,NOT AVAILABLE
1,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,NOT AVAILABLE,19,2,465,3607,4072,1,2079,919,NOT AVAILABLE
2,102030,Bishop State Community College,AL,2,2-year public,-9.802458e+06,3.593069e+06,27,102030,BISHOP STATE COMMUNITY COLLEGE,...,NOT AVAILABLE,8,2,1421,755,2176,2,-999,290,NOT AVAILABLE
3,101240,Gadsden State Community College,AL,2,2-year public,-9.572508e+06,4.027710e+06,3851,101240,GADSDEN STATE COMMUNITY COLLEGE,...,GSCC | GADSDEN STATE | GADSDEN COMMUNITY COLLEGE,8,2,2077,1916,3993,1,110,538,10812401
4,102313,H. Councill Trenholm State Community\n College,AL,2,2-year public,-9.611665e+06,3.809356e+06,625,102313,H COUNCILL TRENHOLM STATE COMMUNITY COLLEGE,...,TRENHOLM STATE,8,2,884,642,1526,2,-999,188,NOT AVAILABLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,234164,Virginia Union University,VA,3,4-year private,-8.621738e+06,4.517615e+06,4393,234164,VIRGINIA UNION UNIVERSITY,...,VUU,21,2,156,1360,1516,1,693,359,NOT AVAILABLE
97,234137,Virginia University of Lynchburg,VA,3,4-year private,-8.811107e+06,4.494328e+06,3317,234137,VIRGINIA UNIVERSITY OF LYNCHBURG,...,NOT AVAILABLE,24,1,36,208,244,1,168,64,NOT AVAILABLE
98,237215,Bluefield State College,WV,1,4-year public,-9.043488e+06,4.476211e+06,2521,237215,BLUEFIELD STATE COLLEGE,...,NOT AVAILABLE,22,2,294,949,1243,2,-999,195,NOT AVAILABLE
99,237899,West Virginia State University,WV,1,4-year public,-9.102374e+06,4.633105e+06,5474,237899,WEST VIRGINIA STATE UNIVERSITY,...,WVSU STATE,21,2,2078,1560,3638,1,369,342,NOT AVAILABLE


In [5]:
hbcu_locs=folium.Map(location=[30.4275125,	-84.28844136], zoom_start=5, min_zoom=4, max_zoom=25)
for row_index, row_values in hbcu_merged.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = str(row_values['Institution'])
    icon=folium.Icon(color="black",icon="fa-solid fa-building-columns" , prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
        icon = icon) 

    marker.add_to(hbcu_locs)
#hbcu_locs.save('../maps/hbcu_locs.html')

hbcu_locs

In [6]:
rental_df=pd.read_csv('../data/Zip_zori_uc_sfrcondomfr_sm_month.csv')
rental_df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1498.067627,...,1907.901834,1881.314050,1889.710273,1896.236122,1908.477009,1914.644524,1917.585619,1912.945631,1893.657670,1884.720629
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1290.517270,...,1802.117493,1805.319508,1809.839779,1821.434627,1830.256460,1842.868613,1845.389752,1831.384137,1821.027073,1822.782122
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,NaN,...,1844.149151,1857.006307,1875.513629,1880.537123,1876.899966,1881.082285,1877.787889,1906.140766,1894.876485,1901.791063
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1422.830222,1412.049962,1417.900245,1421.589198,1445.320002,1442.986026,1449.583812,1454.557440,1459.307173,1473.643939
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,2815.933832,2841.971416,2887.086805,2928.316984,2954.890393,2981.910354,2998.924787,2987.783932,2961.286582,2932.436465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,72017,30490,32461,zip,FL,FL,Panama City Beach,"Crestview-Fort Walton Beach-Destin, FL",Walton County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3377.500000
6238,58956,30490,2876,zip,RI,RI,North Smithfield,"Providence-Warwick, RI-MA",Providence County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2294.042802,2233.333333
6239,91179,30490,76005,zip,TX,TX,Arlington,"Dallas-Fort Worth-Arlington, TX",Tarrant County,NaN,...,1968.894812,2065.454238,2079.714043,2117.915080,2093.434729,2092.953072,2032.127375,1980.655067,1919.732635,1922.238095
6240,61618,30490,10004,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,NaN,...,4930.587046,5115.421314,5142.386670,5201.472103,5165.409957,5259.677877,5264.470039,5322.686348,5207.091313,4972.291667


In [7]:
rental_slice=['RegionName','City','2022-10-31']
rental_df_10_2022=rental_df[rental_slice]
rental_df_10_2022['City']=rental_df_10_2022['City'].str.upper()
rental_df_10_2022

C:\Users\joyce\AppData\Local\Temp\ipykernel_34368\1293038318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental_df_10_2022['City']=rental_df_10_2022['City'].str.upper()


,RegionName,City,2022-10-31
0,77494,KATY,1873.018235
1,77449,KATY,1834.287621
2,77084,HOUSTON,1817.108206
3,79936,EL PASO,1372.593706
4,11385,NEW YORK,2934.748578
...,...,...,...
6237,32461,PANAMA CITY BEACH,NaN
6238,2876,NORTH SMITHFIELD,NaN
6239,76005,ARLINGTON,NaN
6240,10004,NEW YORK,5050.056942


In [8]:
rental_school = pd.merge(hbcu_merged, rental_df_10_2022,  left_on='CITY', right_on='City', how='left')
rental_school

,Unnamed: 0,Institution,State,Designation,Designation.1,X,Y,OBJECTID,IPEDSID,NAME,...,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID,RegionName,City,2022-10-31
0,100654,Alabama A&M University,AL,1,4-year public,-9.636762e+06,4.134481e+06,5132,100654,ALABAMA A & M UNIVERSITY,...,728,5249,5977,1,3220,918,NOT AVAILABLE,61761.0,NORMAL,1023.592059
1,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,465,3607,4072,1,2079,919,NOT AVAILABLE,36117.0,MONTGOMERY,1392.197988
2,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,465,3607,4072,1,2079,919,NOT AVAILABLE,36116.0,MONTGOMERY,1136.480300
3,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,465,3607,4072,1,2079,919,NOT AVAILABLE,77316.0,MONTGOMERY,NaN
4,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,465,3607,4072,1,2079,919,NOT AVAILABLE,77356.0,MONTGOMERY,1848.948947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,234137,Virginia University of Lynchburg,VA,3,4-year private,-8.811107e+06,4.494328e+06,3317,234137,VIRGINIA UNIVERSITY OF LYNCHBURG,...,36,208,244,1,168,64,NOT AVAILABLE,24503.0,LYNCHBURG,NaN
1031,234137,Virginia University of Lynchburg,VA,3,4-year private,-8.811107e+06,4.494328e+06,3317,234137,VIRGINIA UNIVERSITY OF LYNCHBURG,...,36,208,244,1,168,64,NOT AVAILABLE,24504.0,LYNCHBURG,NaN
1032,237215,Bluefield State College,WV,1,4-year public,-9.043488e+06,4.476211e+06,2521,237215,BLUEFIELD STATE COLLEGE,...,294,949,1243,2,-999,195,NOT AVAILABLE,24701.0,BLUEFIELD,NaN
1033,237899,West Virginia State University,WV,1,4-year public,-9.102374e+06,4.633105e+06,5474,237899,WEST VIRGINIA STATE UNIVERSITY,...,2078,1560,3638,1,369,342,NOT AVAILABLE,NaN,NaN,NaN


In [9]:
rental_school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 53 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1035 non-null   int64  
 1   Institution    1035 non-null   object 
 2   State          1035 non-null   object 
 3   Designation    1035 non-null   int64  
 4   Designation.1  1035 non-null   object 
 5   X              1035 non-null   float64
 6   Y              1035 non-null   float64
 7   OBJECTID       1035 non-null   int64  
 8   IPEDSID        1035 non-null   int64  
 9   NAME           1035 non-null   object 
 10  ADDRESS        1035 non-null   object 
 11  CITY           1035 non-null   object 
 12  STATE          1035 non-null   object 
 13  ZIP            1035 non-null   int64  
 14  ZIP4           1035 non-null   object 
 15  TELEPHONE      1035 non-null   object 
 16  TYPE           1035 non-null   int64  
 17  STATUS         1035 non-null   object 
 18  POPULATI

In [10]:
school_average_rent = rental_school.groupby(rental_school.columns[0])[rental_school.columns[52]].mean()
school_average_rent= school_average_rent.to_frame()

In [11]:
school_average_rent.reset_index(inplace=True)
school_average_rent

,Unnamed: 0,2022-10-31
0,100654,1023.592059
1,100724,1309.279695
2,101240,NaN
3,101462,1380.684672
4,101569,1349.919604
...,...,...
96,237215,NaN
97,237899,NaN
98,243665,NaN
99,440916,1310.012023


In [12]:
avg_rental_school = pd.merge(hbcu_merged, school_average_rent,  on= 'Unnamed: 0', how='left')
avg_rental_school

,Unnamed: 0,Institution,State,Designation,Designation.1,X,Y,OBJECTID,IPEDSID,NAME,...,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID,2022-10-31
0,100654,Alabama A&M University,AL,1,4-year public,-9.636762e+06,4.134481e+06,5132,100654,ALABAMA A & M UNIVERSITY,...,18,3,728,5249,5977,1,3220,918,NOT AVAILABLE,1023.592059
1,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,19,2,465,3607,4072,1,2079,919,NOT AVAILABLE,1309.279695
2,102030,Bishop State Community College,AL,2,2-year public,-9.802458e+06,3.593069e+06,27,102030,BISHOP STATE COMMUNITY COLLEGE,...,8,2,1421,755,2176,2,-999,290,NOT AVAILABLE,1057.500853
3,101240,Gadsden State Community College,AL,2,2-year public,-9.572508e+06,4.027710e+06,3851,101240,GADSDEN STATE COMMUNITY COLLEGE,...,8,2,2077,1916,3993,1,110,538,10812401,NaN
4,102313,H. Councill Trenholm State Community\n College,AL,2,2-year public,-9.611665e+06,3.809356e+06,625,102313,H COUNCILL TRENHOLM STATE COMMUNITY COLLEGE,...,8,2,884,642,1526,2,-999,188,NOT AVAILABLE,1309.279695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,234164,Virginia Union University,VA,3,4-year private,-8.621738e+06,4.517615e+06,4393,234164,VIRGINIA UNION UNIVERSITY,...,21,2,156,1360,1516,1,693,359,NOT AVAILABLE,1791.398801
97,234137,Virginia University of Lynchburg,VA,3,4-year private,-8.811107e+06,4.494328e+06,3317,234137,VIRGINIA UNIVERSITY OF LYNCHBURG,...,24,1,36,208,244,1,168,64,NOT AVAILABLE,1232.843710
98,237215,Bluefield State College,WV,1,4-year public,-9.043488e+06,4.476211e+06,2521,237215,BLUEFIELD STATE COLLEGE,...,22,2,294,949,1243,2,-999,195,NOT AVAILABLE,NaN
99,237899,West Virginia State University,WV,1,4-year public,-9.102374e+06,4.633105e+06,5474,237899,WEST VIRGINIA STATE UNIVERSITY,...,21,2,2078,1560,3638,1,369,342,NOT AVAILABLE,NaN


In [13]:
avg_rental_school['2022-10-31'] = avg_rental_school['2022-10-31'].round(2)
avg_rental_school

,Unnamed: 0,Institution,State,Designation,Designation.1,X,Y,OBJECTID,IPEDSID,NAME,...,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID,2022-10-31
0,100654,Alabama A&M University,AL,1,4-year public,-9.636762e+06,4.134481e+06,5132,100654,ALABAMA A & M UNIVERSITY,...,18,3,728,5249,5977,1,3220,918,NOT AVAILABLE,1023.59
1,100724,Alabama State University,AL,1,4-year public,-9.606391e+06,3.811228e+06,2528,100724,ALABAMA STATE UNIVERSITY,...,19,2,465,3607,4072,1,2079,919,NOT AVAILABLE,1309.28
2,102030,Bishop State Community College,AL,2,2-year public,-9.802458e+06,3.593069e+06,27,102030,BISHOP STATE COMMUNITY COLLEGE,...,8,2,1421,755,2176,2,-999,290,NOT AVAILABLE,1057.50
3,101240,Gadsden State Community College,AL,2,2-year public,-9.572508e+06,4.027710e+06,3851,101240,GADSDEN STATE COMMUNITY COLLEGE,...,8,2,2077,1916,3993,1,110,538,10812401,NaN
4,102313,H. Councill Trenholm State Community\n College,AL,2,2-year public,-9.611665e+06,3.809356e+06,625,102313,H COUNCILL TRENHOLM STATE COMMUNITY COLLEGE,...,8,2,884,642,1526,2,-999,188,NOT AVAILABLE,1309.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,234164,Virginia Union University,VA,3,4-year private,-8.621738e+06,4.517615e+06,4393,234164,VIRGINIA UNION UNIVERSITY,...,21,2,156,1360,1516,1,693,359,NOT AVAILABLE,1791.40
97,234137,Virginia University of Lynchburg,VA,3,4-year private,-8.811107e+06,4.494328e+06,3317,234137,VIRGINIA UNIVERSITY OF LYNCHBURG,...,24,1,36,208,244,1,168,64,NOT AVAILABLE,1232.84
98,237215,Bluefield State College,WV,1,4-year public,-9.043488e+06,4.476211e+06,2521,237215,BLUEFIELD STATE COLLEGE,...,22,2,294,949,1243,2,-999,195,NOT AVAILABLE,NaN
99,237899,West Virginia State University,WV,1,4-year public,-9.102374e+06,4.633105e+06,5474,237899,WEST VIRGINIA STATE UNIVERSITY,...,21,2,2078,1560,3638,1,369,342,NOT AVAILABLE,NaN


In [14]:
avg_rental_school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 51 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     101 non-null    int64  
 1   Institution    101 non-null    object 
 2   State          101 non-null    object 
 3   Designation    101 non-null    int64  
 4   Designation.1  101 non-null    object 
 5   X              101 non-null    float64
 6   Y              101 non-null    float64
 7   OBJECTID       101 non-null    int64  
 8   IPEDSID        101 non-null    int64  
 9   NAME           101 non-null    object 
 10  ADDRESS        101 non-null    object 
 11  CITY           101 non-null    object 
 12  STATE          101 non-null    object 
 13  ZIP            101 non-null    int64  
 14  ZIP4           101 non-null    object 
 15  TELEPHONE      101 non-null    object 
 16  TYPE           101 non-null    int64  
 17  STATUS         101 non-null    object 
 18  POPULATION

In [15]:
hbcu_locs2 = folium.Map(location=[30.4275125, -84.28844136], zoom_start=5, min_zoom=4, max_zoom=25)

for row_index, row_values in avg_rental_school.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = f"{row_values['Institution']}: ${row_values['2022-10-31']}"
    
    if pd.notnull(row_values['2022-10-31']):
        icon = folium.Icon(color="darkgreen", icon="fa-solid fa-house", prefix='fa')        
        marker = folium.Marker(
            location=loc, 
            popup=pop,
            icon=icon)
        marker.add_to(hbcu_locs2)

hbcu_locs2

In [16]:
hbcu_locs3 = folium.Map(location=[30.4275125, -84.28844136], zoom_start=5, min_zoom=4, max_zoom=25)

for row_index, row_values in avg_rental_school.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = f"{row_values['Institution']}"
    
    if pd.isnull(row_values['2022-10-31']):
        icon = folium.Icon(color="black", icon="fa-solid fa-house", prefix='fa')        
        marker = folium.Marker(
            location=loc, 
            popup=pop,
            icon=icon)
        marker.add_to(hbcu_locs3)

hbcu_locs3

In [17]:
missing_rental = pd.read_csv('../data/missing_rental_data.csv')
missing_rental

,NAME,2022-10-31
0,ARKANSAS BAPTIST COLLEGE,1026.374915
1,CENTRAL STATE UNIVERSITY,1386.268766
2,CHEYNEY UNIVERSITY OF PENNSYLVANIA,1830.192074
3,FORT VALLEY STATE UNIVERSITY,1347.630772
4,GRAMBLING STATE UNIVERSITY,899.961751
5,JACKSON STATE UNIVERSITY,1250.386141
6,JARVIS CHRISTIAN COLLEGE,1206.029941
7,LANGSTON UNIVERSITY,1473.942248
8,LINCOLN UNIVERSITY,1871.426587
9,PAUL QUINN COLLEGE,1832.831431


In [18]:
missing_rental_loc= pd.merge(missing_rental, hbcu_merged,  on='NAME' , how='left')
missing_rental_loc['2022-10-31']= missing_rental_loc['2022-10-31'].round(2)
missing_rental_loc

,NAME,2022-10-31,Unnamed: 0,Institution,State,Designation,Designation.1,X,Y,OBJECTID,...,ALIAS,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMP,SHELTER_ID
0,ARKANSAS BAPTIST COLLEGE,1026.37,106306,Arkansas Baptist College,AR,3,4-year private,-1.027373e+07,4.127911e+06,5530,...,NOT AVAILABLE,23,1,181,287,468,1,470,112,NOT AVAILABLE
1,CENTRAL STATE UNIVERSITY,1386.27,201690,Central State University,OH,1,4-year public,-9.337122e+06,4.824825e+06,1138,...,NOT AVAILABLE,22,2,1879,2142,4021,1,1437,472,NOT AVAILABLE
2,CHEYNEY UNIVERSITY OF PENNSYLVANIA,1830.19,211608,Cheyney University of Pennsylvania,PA,1,4-year public,-8.407892e+06,4.856108e+06,413,...,NOT AVAILABLE,21,1,64,559,623,1,560,136,NOT AVAILABLE
3,FORT VALLEY STATE UNIVERSITY,1347.63,139719,Fort Valley State University,GA,1,4-year public,-9.339210e+06,3.834162e+06,5578,...,NOT AVAILABLE,20,2,548,2531,3079,1,1400,613,NOT AVAILABLE
4,GRAMBLING STATE UNIVERSITY,899.96,159009,Grambling State University,LA,1,4-year public,-1.032125e+07,3.832121e+06,2379,...,"GSU, GRAMBLING",18,3,1169,4269,5438,1,2500,445,NOT AVAILABLE
5,JACKSON STATE UNIVERSITY,1250.39,175856,Jackson State University,MS,1,4-year public,-1.004176e+07,3.802331e+06,5001,...,NOT AVAILABLE,16,3,1814,5107,6921,1,2464,1168,NOT AVAILABLE
6,JARVIS CHRISTIAN COLLEGE,1206.03,225885,Jarvis Christian University,TX,3,4-year private,-1.059514e+07,3.840778e+06,4046,...,THE BULLDOGS,22,1,245,474,719,1,797,184,NOT AVAILABLE
7,LANGSTON UNIVERSITY,1473.94,207209,Langston University,OK,1,4-year public,-1.082696e+07,4.292912e+06,5443,...,NOT AVAILABLE,19,2,205,1833,2038,1,1200,336,NOT AVAILABLE
8,LINCOLN UNIVERSITY,1871.43,177940,Lincoln University,MO,1,4-year public,-1.026014e+07,4.659516e+06,6058,...,LINCOLN UNIVERSITY OF MISSOURI,20,2,658,1354,2012,1,1100,464,NOT AVAILABLE
9,LINCOLN UNIVERSITY,1871.43,213598,Lincoln University,PA,1,4-year public,-8.452297e+06,4.838041e+06,3764,...,NOT AVAILABLE,19,2,260,1817,2077,1,1800,437,NOT AVAILABLE


In [19]:
hbcu_locs4 = folium.Map(location=[30.4275125, -84.28844136], zoom_start=5, min_zoom=4, max_zoom=25)

for row_index, row_values in missing_rental_loc.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = f"{row_values['NAME']}: ${row_values['2022-10-31']}"
    
    if pd.notnull(row_values['2022-10-31']):
        icon = folium.Icon(color="darkgreen", icon="fa-solid fa-house", prefix='fa')        
        marker = folium.Marker(
            location=loc, 
            popup=pop,
            icon=icon)
        marker.add_to(hbcu_locs4)

hbcu_locs4